# More in Depth Example on Functional API

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras import layers, regularizers
from keras.datasets import mnist
import pandas as pd
import os

### Hyperparameters

In [ ]:
BATCH_SIZE = 64
WEIGHT_DECAY = 0.001
LEARNING_RATE = 0.001

### Dataset
【🔔】[**Download**](https://www.kaggle.com/datasets/eb9594e5b728b2eb74ff8d5e57a9b74634330bfa79d9195d6ebdc7745b9802c3)

In [ ]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

train_images = os.getcwd() + "/train_images/" + train_df.iloc[:,0].values
test_images = os.getcwd() + "/test_images/" + test_df.iloc[:,0].values

train_labels = train_df.iloc[:,1:].values
test_labels = test_df.iloc[:,1:].values

In [ ]:
def read_images(images_path, label):
    image = tf.io.read_file(images_path)
    image = tf.image.decode_image(image, channels=1, dtype=tf.float32)
    
    # image.set_shape((64, 64, 1))
    # label[0].set_shape([])
    # label[1].set_shape([])
    
    labels = {"first_num":label[0], "second_num":label[1]}
    return image, labels

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
train_dataset = tf.data.Dataset.from_tensor_slices(
    (train_images, train_labels)
)
train_dataset = (
    train_dataset.shuffle(buffer_size=len(train_labels))
    .map(read_images)
    .batch(batch_size=BATCH_SIZE)
    .prefetch(buffer_size=AUTOTUNE)
)
test_dataset = tf.data.Dataset.from_tensor_slices(
    (test_images, test_labels)
)
test_dataset = (
    train_dataset
    .map(read_images)
    .batch(batch_size=BATCH_SIZE)
    .prefetch(buffer_size=AUTOTUNE)
)

### Model

In [ ]:
inputs = keras.Input(shape=(64, 64, 3))
x = layers.Conv2D(
    filters = 32,
    kernel_size = 3,
    padding = 'same',
    kernel_regularizer = regularizers.l2(WEIGHT_DECAY)
)(inputs)
x = layers.BatchNormalization()(x)
x = keras.activations.relu(x)
x = layers.Conv2D(64, 3, kernel_regularizer = regularizers.l2(WEIGHT_DECAY))(x)
x = layers.BatchNormalization()(x)
x = keras.activations.relu(x)
x = layers.MaxPooling2D()(x)
x = layers.Conv2D(64, 3, activation='relu',
                  kernel_regularizer = regularizers.l2(WEIGHT_DECAY))(x)
x = layers.Conv2D(128, 3, activation='relu')(x)
x = layers.MaxPooling2D()(x)
x = layers.Flatten()(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(64, activation='relu')(x)
output1 = layers.Dense(10, activation='softmax', name='first_num')(x)
output2 = layers.Dense(10, activation='softmax', name='second_num')(x)
model = keras.Model(inputs = inputs, outputs = [output1, output2])

In [ ]:
model.compile(
    loss=[
            keras.losses.SparseCategoricalCrossentropy(),
            keras.losses.SparseCategoricalCrossentropy()
        ],
    optimizer=keras.optimizers.Adam(learning_rate=LEARNING_RATE),
    metrics=["accuracy"]
)

model.fit(train_dataset, epochs=5, verbose=2)
model.evaluate(test_dataset, verbose=2)